In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from geojson import GeoJSON
import json
import matplotlib
from numpy import mean, all, median

from bokeh.models import (ColumnDataSource, GMapPlot, Range1d, ColumnDataSource, LinearAxis,
HoverTool, PanTool, WheelZoomTool, BoxZoomTool, ResetTool,CustomJS,
GMapOptions, NumeralTickFormatter, PrintfTickFormatter, MapOptions)
from bokeh.models.glyphs import Patches
from bokeh.plotting import figure, save, show
from bokeh.resources import CDN
from bokeh.io import output_file, show, saving
from bokeh.models.widgets import RadioButtonGroup
from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import Panel, Tabs
from bokeh import tile_providers
from bokeh.tile_providers import CARTODBPOSITRON
import pyproj

In [5]:
data = gpd.read_file('/Users/christophermarker/Documents/DSI7/Data/Capstone/ne_50m_admin_0_countries')

In [6]:
data.head()

,scalerank,featurecla,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,REGION_WB,NAME_LEN,LONG_LEN,ABBREV_LEN,TINY,HOMEPART,MIN_ZOOM,MIN_LABEL,MAX_LABEL,geometry
0,3,Admin-0 country,5.0,Netherlands,NL1,1.0,2.0,Country,Aruba,ABW,...,Latin America & Caribbean,5.0,5.0,5.0,4.0,-99.0,0.0,5.0,10.0,"POLYGON ((-69.89912109375 12.45200195312499, -..."
1,1,Admin-0 country,3.0,Afghanistan,AFG,0.0,2.0,Sovereign country,Afghanistan,AFG,...,South Asia,11.0,11.0,4.0,-99.0,1.0,0.0,3.0,7.0,"POLYGON ((74.52646484375001 37.03066406249999,..."
2,1,Admin-0 country,3.0,Angola,AGO,0.0,2.0,Sovereign country,Angola,AGO,...,Sub-Saharan Africa,6.0,6.0,4.0,-99.0,1.0,0.0,3.0,7.0,(POLYGON ((14.39863281250001 -5.89267578125000...
3,1,Admin-0 country,6.0,United Kingdom,GB1,1.0,2.0,Dependency,Anguilla,AIA,...,Latin America & Caribbean,8.0,8.0,4.0,-99.0,-99.0,0.0,5.0,10.0,"POLYGON ((-63.001220703125 18.22177734374999, ..."
4,1,Admin-0 country,6.0,Albania,ALB,0.0,2.0,Sovereign country,Albania,ALB,...,Europe & Central Asia,7.0,7.0,4.0,-99.0,1.0,0.0,5.0,10.0,"POLYGON ((20.06396484375 42.54726562499999, 20..."


In [19]:
wgs84=pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
webdb=pyproj.Proj("+init=EPSG:3857") # Web Mercator
robinson = pyproj.Proj('+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')

In [9]:
def convert_to_bokeh(shape):
    polys = list(shape.geometry.values)
    x = []
    y = []
    for p in polys:
        x_list = []
        y_list = []
        for t in mapping(p)['coordinates'][0]:
            x1, y1 = pyproj.transform(wgs84, robinson, t[0], t[1])

            x_list.append(x1)
            y_list.append(y1)
        x.append(x_list)
        y.append(y_list)
    frame = shape.copy()
    frame['X'] = x
    frame['Y'] = y    
    return frame

data2 = convert_to_bokeh(data)

In [10]:
data2.head()

,scalerank,featurecla,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,LONG_LEN,ABBREV_LEN,TINY,HOMEPART,MIN_ZOOM,MIN_LABEL,MAX_LABEL,geometry,X,Y
0,3,Admin-0 country,5.0,Netherlands,NL1,1.0,2.0,Country,Aruba,ABW,...,5.0,5.0,4.0,-99.0,0.0,5.0,10.0,"POLYGON ((-69.89912109375 12.45200195312499, -...","[-7781134.567053617, -7780754.08051282, -77859...","[1397192.8510201334, 1393886.5572352135, 13956..."
1,1,Admin-0 country,3.0,Afghanistan,AFG,0.0,2.0,Sovereign country,Afghanistan,AFG,...,11.0,4.0,-99.0,1.0,0.0,3.0,7.0,"POLYGON ((74.52646484375001 37.03066406249999,...","[8296248.117029057, 8297911.38676454, 8285627....","[4443381.824425247, 4442197.166817215, 4436833..."
2,1,Admin-0 country,3.0,Angola,AGO,0.0,2.0,Sovereign country,Angola,AGO,...,6.0,4.0,-99.0,1.0,0.0,3.0,7.0,(POLYGON ((14.39863281250001 -5.89267578125000...,"[(1602848.4728068206, -655969.6673786066)]","[(1649806.7794535276, -656702.9246533032)]"
3,1,Admin-0 country,6.0,United Kingdom,GB1,1.0,2.0,Dependency,Anguilla,AIA,...,8.0,4.0,-99.0,-99.0,0.0,5.0,10.0,"POLYGON ((-63.001220703125 18.22177734374999, ...","[-7013263.808026521, -7030940.125607561, -7030...","[2063523.5948489928, 2057618.8433169818, 20610..."
4,1,Admin-0 country,6.0,Albania,ALB,0.0,2.0,Sovereign country,Albania,ALB,...,7.0,4.0,-99.0,1.0,0.0,5.0,10.0,"POLYGON ((20.06396484375 42.54726562499999, 20...","[2233510.3497003913, 2237913.1225296203, 22470...","[5243312.990987732, 5239897.59306411, 5224989...."


In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON

output_file("tile.html")

# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(CARTODBPOSITRON)

show(p)

In [ ]:
robinson = pyproj.Proj('+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs')

In [25]:
source2 = ColumnDataSource(data2)



x, y = pyproj.transform(wgs84, webdb, 0.0, 0.0)
bound = 3000 # 45 km

#x_scroll = Range1d(x-bound,x+bound, bounds=(x-bound*1.25,x+bound*1.25)) # different bounds
#y_scroll = Range1d(y-bound,y+bound, bounds=(y-bound*1.25,y+bound*1.25)) # different bounds


plot = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
    toolbar_location="right", plot_width=800,
    plot_height=800)


#wheel = plot.toolbar.tools[1] # select the wheel tool by index rather then making a new one
#plot.toolbar.active_scroll = wheel
plot.add_tile(CARTODBPOSITRON)

tracts = Patches(xs='X', ys='Y',
fill_color="white", fill_alpha=0.4, 
line_color="black", line_width=0.10)

plot.add_glyph(source2, tracts)


show(plot)

ValueError: Out of range float values are not JSON compliant

In [23]:
data2.head()

,scalerank,featurecla,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,LONG_LEN,ABBREV_LEN,TINY,HOMEPART,MIN_ZOOM,MIN_LABEL,MAX_LABEL,geometry,X,Y
0,3,Admin-0 country,5.0,Netherlands,NL1,1.0,2.0,Country,Aruba,ABW,...,5.0,5.0,4.0,-99.0,0.0,5.0,10.0,"POLYGON ((-69.89912109375 12.45200195312499, -...","[-7781134.567053617, -7780754.08051282, -77859...","[1397192.8510201334, 1393886.5572352135, 13956..."
1,1,Admin-0 country,3.0,Afghanistan,AFG,0.0,2.0,Sovereign country,Afghanistan,AFG,...,11.0,4.0,-99.0,1.0,0.0,3.0,7.0,"POLYGON ((74.52646484375001 37.03066406249999,...","[8296248.117029057, 8297911.38676454, 8285627....","[4443381.824425247, 4442197.166817215, 4436833..."
2,1,Admin-0 country,3.0,Angola,AGO,0.0,2.0,Sovereign country,Angola,AGO,...,6.0,4.0,-99.0,1.0,0.0,3.0,7.0,(POLYGON ((14.39863281250001 -5.89267578125000...,"[(1602848.4728068206, -655969.6673786066)]","[(1649806.7794535276, -656702.9246533032)]"
3,1,Admin-0 country,6.0,United Kingdom,GB1,1.0,2.0,Dependency,Anguilla,AIA,...,8.0,4.0,-99.0,-99.0,0.0,5.0,10.0,"POLYGON ((-63.001220703125 18.22177734374999, ...","[-7013263.808026521, -7030940.125607561, -7030...","[2063523.5948489928, 2057618.8433169818, 20610..."
4,1,Admin-0 country,6.0,Albania,ALB,0.0,2.0,Sovereign country,Albania,ALB,...,7.0,4.0,-99.0,1.0,0.0,5.0,10.0,"POLYGON ((20.06396484375 42.54726562499999, 20...","[2233510.3497003913, 2237913.1225296203, 22470...","[5243312.990987732, 5239897.59306411, 5224989...."
